In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install kaleido

     |████████████████████████████████| 79.9 MB 220 kB/s            


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go
import glob
import os

In [4]:
def create_dir (direct):
  if not os.path.exists(direct):
    os.mkdir(direct)

In [5]:
ROOT_DIR = "/kaggle/input/ferset" #MT Directory
DATASET_02 = ROOT_DIR + "/02_FER"
DATASET_02_TRAIN = DATASET_02 + "/train"
DATASET_02_TEST = DATASET_02+ "/test"

In [6]:
import PIL

image = PIL.Image.open(DATASET_02_TRAIN + "/surprise/Training_99984132.jpg")
#image to open

width, height = image.size

print(image.mode)
print(width, height)

L
48 48


In [7]:
datagen = ImageDataGenerator()
image_size = (48, 48)
colormode = 'rgb'
test_set = datagen.flow_from_directory(DATASET_02_TEST, target_size=image_size, color_mode =colormode, batch_size=32, shuffle=False)
train_set = datagen.flow_from_directory(DATASET_02_TRAIN, target_size=image_size, color_mode =colormode, batch_size=32)

Found 7178 images belonging to 7 classes.
Found 28709 images belonging to 7 classes.


In [8]:
def create_model_V1():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(16, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))
    
    return model


model_1 = create_model_V1()
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               3277056   
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 1

2022-05-08 03:47:01.739597: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
def get_data_df(dir):
  # modified code from: https://www.kaggle.com/namgalielei/simple-load-images-and-count-number-of-each-class

  train_df = pd.DataFrame()

  trainset = glob.glob(dir)
  #print(trainset)
  train_df['filename'] = [img.split("/")[-2]+"/"+img.split("/")[-1] for img in trainset]
  train_df['label'] = [img.split("/")[-2] for img in trainset]

  return train_df

In [10]:
test_train = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
test_train

,filename,label
0,surprise/Training_66056468.jpg,surprise
1,surprise/Training_26522394.jpg,surprise
2,surprise/Training_63239185.jpg,surprise
3,surprise/Training_67725241.jpg,surprise
4,surprise/Training_71308969.jpg,surprise
...,...,...
28704,happy/Training_18866880.jpg,happy
28705,happy/Training_16513509.jpg,happy
28706,happy/Training_10834907.jpg,happy
28707,happy/Training_30459679.jpg,happy


In [11]:
total = 0
value_list = []
for item in os.listdir(DATASET_02_TRAIN):
    y = len(os.listdir(DATASET_02_TRAIN+'/'+item))+len(os.listdir(DATASET_02_TEST+'/'+item))
    value_list.append(y)
    total += y
    print(item)
print(total)
print(value_list)
weights = {}
for n in range(0,len(value_list)):
    w = total/(len(value_list)*value_list[n])
    weights[n] = w
    
print(weights)

surprise
fear
angry
neutral
sad
disgust
happy
35887
[4002, 5121, 4953, 6198, 6077, 547, 8989]
{0: 1.2810380524023703, 1: 1.001115853488437, 2: 1.035072539009547, 3: 0.8271562255105334, 4: 0.8436258492207152, 5: 9.372420997649517, 6: 0.5703319930708962}


In [12]:
create_dir('/kaggle/working/Saved_CNN_Models')


In [13]:
from sklearn.model_selection import KFold, StratifiedKFold

train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
Y = train_data[['label']]

#Y = final_train[['label']]
#n = len(final_train)   

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

#Un needed preprocessing aside from rescale
#idg = ImageDataGenerator(rescale=1./255)
create_dir('/kaggle/working/Saved_CNN_Models')
#model name
def get_model_name(k):
    return 'model_'+str(k)+'.h5'
def model_test(fold_count, epoch, model_method, image_size, mode):
    skf = StratifiedKFold(n_splits = fold_count, random_state = 7, shuffle = True)
    train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
      #Incorperate preprocessing in the method 
      #####################################################################
      #aug_train = get_data_df(DATASET_01_ROOT+"/Aug_train/*/*.jpg")
      #aug_train['filename'] = aug_train['filename'].str[7:]
      #final_train = test_train.append(aug_train, sort=False, ignore_index=True)


      #######################################################################
      #idg = ImageDataGenerator(rescale=1./255)
    idg = ImageDataGenerator()
    Y = train_data[['label']]
    n = len(train_data)    
    VALIDATION_ACCURACY = []
    VALIDATION_LOSS = []

      #create a directory
    save_dir = '/kaggle/working/Saved_CNN_Models/'+model_method.__name__+'_'+str(epoch)+'epochs/'
    create_dir(save_dir)
    
    img_dir = DATASET_02_TRAIN

    fold_var = 1

      #n is number of samples, doesn't care about x values for split, just y
    for train_index, val_index in skf.split(np.zeros(n),Y):

        #Index value for fold
        training_data = train_data.iloc[train_index]
        validation_data = train_data.iloc[val_index]
        #generator for train
        #print("hi")
        #print(validation_data.head())
        #Modify the training size, have an input adjustability 
        train_data_generator = idg.flow_from_dataframe(training_data, directory = img_dir,
                                                       target_size=image_size, batch_size=32, color_mode =mode,
                                                       x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #generator for validation
        valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = img_dir,
                                                        target_size=image_size, color_mode =mode,batch_size=32,
                                                        x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #create a method for this
        # CREATE NEW MODEL
        model = model_method()
        # COMPILE NEW MODEL
        model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])

        # CREATE CALLBACKS
        #Unsure about checpoint
        checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+model_method.__name__+"_"+get_model_name(fold_var), 
                    monitor='val_accuracy', verbose=1, 
                    save_best_only=True, mode='max')

        callbacks_list = [checkpoint]
        # There can be other callbacks, but just showing one because it involves the model name
        # This saves the best model
        # FIT THE MODEL
        #Include the weights for classes
        history = model.fit(train_data_generator,
                epochs=epoch,
                callbacks=callbacks_list,
                validation_data=valid_data_generator,
                class_weight = weights,
                verbose = 2)

        #PLOT HISTORY
        #Plots for whichever
        #		:
        #plt.plot(history.history['binary_accuracy'], label='accuracy')
        #plt.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
        #plt.xlabel('Epoch')
        #plt.ylabel('Accuracy')
        #plt.ylim([0.3, 1])
        #plt.legend(loc='lower right')
        #plt.show()
        #frame = {'Accuracy': history.history['binary_accuracy'], 'Val_Accuracy': history.history['val_binary_accuracy']}
        #fig = px.line(frame)
        #fig.show()
        fig = go.Figure()
        fig.add_trace(go.Scatter(y= history.history['accuracy'],
                          mode='lines+markers',
                          name='Training Accuracy'))
        fig.add_trace(go.Scatter(y= history.history['val_accuracy'],
                          mode='lines+markers',
                          name='Validation Accuracy'))
        fig.update_yaxes(range=[0, 1])
        fig.update_layout(
            title="Fold "+ str(fold_var)+ "'s performance",
            title_x=0.5,
            xaxis_title="Epochs",
            yaxis_title="Accuracy",
            autosize=False,
            width=800,
            height=400,
            margin=dict(l=20, r=20, t=40, b=20),)
        fig.show()
        graph_dir = '/kaggle/working/img'
        create_dir(graph_dir)
        graph_dir = graph_dir+'/'+model_method.__name__
        create_dir(graph_dir)
        fig.write_image(graph_dir+'/fold'+str(fold_var)+'.png') 

        #		:
        # LOAD BEST MODEL to evaluate the performance of the model
        model.load_weights(save_dir+model_method.__name__+"_" + "model_"+str(fold_var)+".h5")

        results = model.evaluate(test_set)
        results = dict(zip(model.metrics_names,results))

        VALIDATION_ACCURACY.append(results['accuracy'])
        VALIDATION_LOSS.append(results['loss'])

        tf.keras.backend.clear_session()

        fold_var += 1

In [14]:
#model_test(5, 1, create_model_V1, (48,48), 'rgb')

In [15]:
def transfer_learning_model_resnet_50():

  IMG_SHAPE = (48,48,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.resnet50.preprocess_input(inputs)
  base_model = tf.keras.applications.resnet50.ResNet50(input_shape=(48,48,3,),
                                               include_top=False,
                                               weights='imagenet')
  print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 140
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.4)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_resnet_50 = transfer_learning_model_resnet_50()

94781440/94765736 [==============================] - 1s 0us/step
175


In [16]:
transfer_model_resnet_50.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 48, 48, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 48, 48, 3)         0         
_________________________________________________________________
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 14343 

In [17]:
#model_test(5, 30, transfer_learning_model_resnet_50, (48,48,), 'rgb')

In [18]:
def transfer_learning_model_vgg_16():

  IMG_SHAPE = (48,48,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.vgg16.preprocess_input(inputs)
  base_model = tf.keras.applications.vgg16.VGG16(input_shape=(48,48,3,),
                                               include_top=False,
                                               weights='imagenet')
  print(len(base_model.layers))
  base_model.trainable = False

  fine_tune_at = 14
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.4)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_vgg_16 = transfer_learning_model_vgg_16()

58900480/58889256 [==============================] - 0s 0us/step
19


In [19]:
transfer_model_vgg_16.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem_1 ( (None, 48, 48, 3)         0         
_________________________________________________________________
tf.nn.bias_add_1 (TFOpLambda (None, 48, 48, 3)         0         
_________________________________________________________________
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 3591

In [20]:
model_test(5, 30, transfer_learning_model_vgg_16, (48,48,), 'rgb')

Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19


2022-05-08 03:47:21.519286: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
718/718 - 311s - loss: 40.5200 - accuracy: 0.1314 - val_loss: 19.2547 - val_accuracy: 0.1639

Epoch 00001: val_accuracy improved from -inf to 0.16388, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_1.h5
Epoch 2/30
718/718 - 303s - loss: 22.0924 - accuracy: 0.1728 - val_loss: 13.4896 - val_accuracy: 0.1832

Epoch 00002: val_accuracy improved from 0.16388 to 0.18321, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_1.h5
Epoch 3/30
718/718 - 307s - loss: 16.2240 - accuracy: 0.1909 - val_loss: 10.5418 - val_accuracy: 0.1966

Epoch 00003: val_accuracy improved from 0.18321 to 0.19662, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_1.h5
Epoch 4/30
718/718 - 304s - loss: 12.5810 - accuracy: 0.2059 - val_loss: 8.5080 - val_accuracy: 0.2045

Epoch 00004

225/225 [==============================] - 80s 356ms/step - loss: 4.9307 - accuracy: 0.2320
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/30
718/718 - 307s - loss: 37.2025 - accuracy: 0.1739 - val_loss: 20.5532 - val_accuracy: 0.1677

Epoch 00001: val_accuracy improved from -inf to 0.16771, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_2.h5
Epoch 2/30
718/718 - 307s - loss: 23.9813 - accuracy: 0.1810 - val_loss: 14.9412 - val_accuracy: 0.1823

Epoch 00002: val_accuracy improved from 0.16771 to 0.18234, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_2.h5
Epoch 3/30
718/718 - 314s - loss: 17.6269 - accuracy: 0.1956 - val_loss: 10.9934 - val_accuracy: 0.1966

Epoch 00003: val_accuracy improved from 0.18234 to 0.19662, saving model to /kaggle/work

225/225 [==============================] - 78s 347ms/step - loss: 6.6018 - accuracy: 0.2239
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/30
718/718 - 311s - loss: 28.0338 - accuracy: 0.1663 - val_loss: 16.4251 - val_accuracy: 0.1684

Epoch 00001: val_accuracy improved from -inf to 0.16841, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_3.h5
Epoch 2/30
718/718 - 311s - loss: 19.9272 - accuracy: 0.1856 - val_loss: 12.5461 - val_accuracy: 0.1799

Epoch 00002: val_accuracy improved from 0.16841 to 0.17990, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_3.h5
Epoch 3/30
718/718 - 313s - loss: 14.8268 - accuracy: 0.2005 - val_loss: 9.9797 - val_accuracy: 0.1881

Epoch 00003: val_accuracy improved from 0.17990 to 0.18809, saving model to /kaggle/worki

225/225 [==============================] - 79s 353ms/step - loss: 5.5462 - accuracy: 0.2186
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/30
718/718 - 314s - loss: 34.4255 - accuracy: 0.1503 - val_loss: 18.8265 - val_accuracy: 0.1693

Epoch 00001: val_accuracy improved from -inf to 0.16928, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_4.h5
Epoch 2/30
718/718 - 313s - loss: 20.8758 - accuracy: 0.1773 - val_loss: 12.8494 - val_accuracy: 0.1809

Epoch 00002: val_accuracy improved from 0.16928 to 0.18095, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_4.h5
Epoch 3/30
718/718 - 312s - loss: 15.1414 - accuracy: 0.1968 - val_loss: 9.6978 - val_accuracy: 0.2001

Epoch 00003: val_accuracy improved from 0.18095 to 0.20010, saving model to /kaggle/worki

225/225 [==============================] - 80s 353ms/step - loss: 6.5621 - accuracy: 0.2226
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
19
Epoch 1/30
718/718 - 314s - loss: 35.7245 - accuracy: 0.1590 - val_loss: 18.4770 - val_accuracy: 0.1751

Epoch 00001: val_accuracy improved from -inf to 0.17506, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_5.h5
Epoch 2/30
718/718 - 315s - loss: 22.0709 - accuracy: 0.1789 - val_loss: 13.3037 - val_accuracy: 0.1895

Epoch 00002: val_accuracy improved from 0.17506 to 0.18951, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_5.h5
Epoch 3/30
718/718 - 310s - loss: 16.3888 - accuracy: 0.1894 - val_loss: 10.4729 - val_accuracy: 0.2015

Epoch 00003: val_accuracy improved from 0.18951 to 0.20153, saving model to /kaggle/work